In [ ]:
%%sql
-- Créer une vue temporaire de la table Bronze
CREATE OR REPLACE TEMPORARY VIEW bronze_wind_power AS
SELECT *
FROM `WindPowerAnalytics-2025`.LH_Wind_Power_Bronze.dbo.wind_power;

In [6]:
# 1️⃣ Charger directement le Delta depuis le chemin ABFSS
bronze_table_path = "abfss://c5409968-520e-49dc-87fb-b95c69c590fb@onelake.dfs.fabric.microsoft.com/5b1a29b2-a02e-4c41-9f7c-a2c72c8a6916/Tables/dbo/wind_power"
df_bronze = spark.read.format("delta").load(bronze_table_path)

# 2️⃣ Créer une vue temporaire
df_bronze.createOrReplaceTempView("bronze_wind_power")


StatementMeta(, d4f24f0d-e073-4c65-9b26-5eab5aef57aa, 8, Finished, Available, Finished)

In [7]:
%%sql
-- Nettoyer et enrichir les données
CREATE OR REPLACE TEMPORARY VIEW transformed_wind_power AS
SELECT
    production_id,
    date,
    turbine_name,
    capacity,
    location_name,
    latitude,
    longitude,
    region,
    status,
    responsible_department,
    wind_direction,
    
    -- 🔢 Arrondi des valeurs numériques
    ROUND(wind_speed, 2) AS wind_speed,
    ROUND(energy_produced, 2) AS energy_produced,
    
    -- 📅 Extraction des composants de date
    DAY(date) AS day,
    MONTH(date) AS month,
    QUARTER(date) AS quarter,
    YEAR(date) AS year,
    
    -- 🕐 Correction du format de time
    REGEXP_REPLACE(time, '-', ':') AS time,
    
    -- ⏰ Extraction des composants de temps
    CAST(SUBSTRING(time, 1, 2) AS INT) AS hour_of_day,
    CAST(SUBSTRING(time, 4, 2) AS INT) AS minute_of_hour,
    CAST(SUBSTRING(time, 7, 2) AS INT) AS second_of_minute,
    
    -- 🌅 Calcul de la période de la journée
    CASE
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 5 AND 11 THEN 'Morning'
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 12 AND 16 THEN 'Afternoon'
        WHEN CAST(SUBSTRING(time, 1, 2) AS INT) BETWEEN 17 AND 20 THEN 'Evening'
        ELSE 'Night'
    END AS time_period
    
FROM bronze_wind_power;

StatementMeta(, d4f24f0d-e073-4c65-9b26-5eab5aef57aa, 9, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [8]:
%%sql
-- Supprimer l'ancienne table Silver si elle existe
DROP TABLE IF EXISTS WindPowerAnalytics.LH_Wind_Power_Silver.dbo.wind_power;

StatementMeta(, d4f24f0d-e073-4c65-9b26-5eab5aef57aa, 10, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [11]:
%%sql
-- Créer la nouvelle table Silver avec les données transformées
CREATE TABLE `WindPowerAnalytics-2025`.LH_silver.dbo.wind_power
USING delta
AS
SELECT * FROM transformed_wind_power;


StatementMeta(, d4f24f0d-e073-4c65-9b26-5eab5aef57aa, 13, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [ ]:
%%sql
-- Vérifier que la table a été créée avec succès
SELECT 
    COUNT(*) as total_rows,
    MIN(date) as min_date,
    MAX(date) as max_date,
    COUNT(DISTINCT turbine_name) as turbine_count
FROM `WindPowerAnalytics-2025`.LH_silver.dbo.wind_power;

StatementMeta(, , -1, SessionError, , SessionError)

InvalidHttpRequestToLivy: [TooManyRequestsForCapacity] This spark job can't be run because you have hit a spark compute or API rate limit. To run this spark job, cancel an active Spark job through the Monitoring hub, choose a larger capacity SKU, or try again later. HTTP status code: 430 {Learn more} HTTP status code: 430.

In [ ]:
#voir le schéma de la table
%%sql
-- Afficher le schéma simple de la table
DESCRIBE `WindPowerAnalytics-2025`.LH_silver.dbo.wind_power;


StatementMeta(, , -1, SessionError, , SessionError)

InvalidHttpRequestToLivy: [TooManyRequestsForCapacity] This spark job can't be run because you have hit a spark compute or API rate limit. To run this spark job, cancel an active Spark job through the Monitoring hub, choose a larger capacity SKU, or try again later. HTTP status code: 430 {Learn more} HTTP status code: 430.